# DSPY Learn - Modules

https://dspy.ai/learn/programming/modules/

In [1]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4omin = dspy.LM('gpt-4o-mini', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4omin)

lm = lm_gpt4omin
lm('hello there')


Getting keys from environment variables


['Hello! How can I assist you today?']